In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
from os.path import join
from collections import defaultdict
import numpy as np
import joblib
from tqdm import tqdm
import sys
from copy import deepcopy
sys.path.append('../notebooks')
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import viz
from neurosynth import term_dict, term_dict_rev, get_neurosynth_flatmaps
neurosynth_compare = __import__('04_neurosynth_compare')
import viz
import neurosynth
from cortex import mni
import os
os.environ["FSLDIR"] = "/home/chansingh/fsl"

/home/chansingh/fsl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note, this notebook requires first running `03_export_qa_flatmaps.ipynb` into `df_qa_dict.pkl` files for each subject.

### compute correlations with qa flatmaps and plot avgs

In [2]:
# setting = 'shapley_neurosynth'
# setting = 'full_neurosynth'
# setting = 'individual_gpt4'
# settings = ['full_neurosynth_wordrate']  # shapley_neurosynth, individual_gpt4
settings = ['full_neurosynth']
setting = settings[0]
# subjects = ['UTS01', 'UTS02', 'UTS03']
subjects = [f'UTS0{i}' for i in range(1, 9)]


# comparison hyperparams
apply_mask = True
frac_voxels_to_keep = 0.1  # 0.10
frac_voxels_to_keep_list = [frac_voxels_to_keep]


flatmaps_qa_dicts_by_subject = neurosynth.load_flatmaps_qa_dicts_by_subject(
    subjects, settings)

# flatmaps_gt_dict_list_subject_mni = {subject: [convert_to_mni_space(flatmaps_gt_dict_list_subject[subject][qs[i]], subject=subject)
#                                                for i in tqdm(range(len(qs)))]
#                                      for subject in ['UTS01', 'UTS02', 'UTS03']}
flatmaps_gt_dict_mni = neurosynth.get_neurosynth_flatmaps(mni=True)
qs = list(set(flatmaps_qa_dicts_by_subject['UTS01'].keys()) & set(
    flatmaps_gt_dict_mni.keys()))

100%|██████████| 8/8 [00:00<00:00, 27.38it/s]


In [ ]:
pvals_subject = neurosynth_compare.compute_pvals_for_subject(
    corrs_df, 'UTS01', frac_voxels_to_keep_list)
pvals_subject.style.background_gradient().format(precision=3)

### Re-do analysis in MNI space

In [ ]:
df.style.background_gradient(axis=None, cmap="coolwarm_r", vmin=-
                             df.abs().max().max(), vmax=df.abs().max().max()).format(precision=3)

### View flatmaps in 1 plot

In [ ]:
corrs_df = pd.read_pickle(join(repo_dir, 'qa_results',
                               'neurosynth', setting + '_corrs_df.pkl'))

In [ ]:
setting = 'shapley_neurosynth'
for subject in ['UTS01', 'UTS02', 'UTS03']:
    img_dir1 = join(repo_dir, 'qa_results', 'neurosynth',
                    subject, 'neurosynth')
    img_dir2 = join(repo_dir, 'qa_results', 'neurosynth',
                    subject, setting)

    # read images and combine them with their filenames on a single plot
    # fnames = os.listdir(img_dir1)
    # fnames = [f for f in fnames if f.endswith('.png')]
    # only keep the ones that are in both directories
    # fnames = [f for f in fnames if f in os.listdir(img_dir2)]

    corrs = corrs_df[corrs_df['subject'] == subject]
    # corrs = corrs.sort_values(f'corrs_{frac_voxels_to_keep}', ascending=False)
    fnames = [v + '.png' for v in corrs['questions'].values]

    n = len(fnames)
    C = 4
    R = int(np.ceil(n / C))

    fig, axs = plt.subplots(R, C, figsize=(C * 3.2, R * 1))
    axs = axs.flatten()
    for i in range(len(axs)):
        axs[i].axis('off')
    for i, fname in enumerate(fnames):
        img1 = plt.imread(join(img_dir1, fname))
        img2 = plt.imread(join(img_dir2, fname))
        axs[i].imshow(np.concatenate([img1, img2], axis=1))
        axs[i].set_title(
            f'{term_dict_rev[fname[:-4]]} ({corrs["corrs"].values[i]:0.3f})', fontsize=8)

    # add text in bottom right of figure
    fig.text(0.99, 0.01, f'{subject}\nNeurosynth on left, QA on right',
             ha='right', va='bottom', fontsize=8)
    plt.tight_layout()
    plt.savefig(join(repo_dir, 'qa_results', 'neurosynth',
                subject, f'flatmaps_{setting}_{subject}.png'), dpi=300)
    plt.close()